In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from environment import *

from setting import *

In [ ]:
target_x_sample = pd.read_table(
    '../data/target_x_sample.tsv',
    index_col=0,
)

In [ ]:
gene_x_sample = pd.read_table(
    '../output/gene_x_sample__processed.tsv',
    index_col=0,
)

In [ ]:
output_directory_path = '../output/gene_set'

In [ ]:
data_gene_set_directory_path = '../data/gene_set'

gene_set_file_paths = tuple('{}/{}'.format(
    data_gene_set_directory_path,
    gene_set_file_name,
) for gene_set_file_name in os.listdir(data_gene_set_directory_path) if gene_set_file_name.endswith('.gmt'))

pprint(gene_set_file_paths)

In [ ]:
feature_dicts = {}

In [ ]:
def make_gene_set_x_sample_and_add_to_feature_dicts(
    name,
    gene_sets,
    overwrite,
):
    
    gene_set_x_sample_file_path = '{}/{}_gene_set_x_sample.tsv'.format(
        output_directory_path,
        name,
    )
    
    if overwrite or not os.path.isfile(gene_set_x_sample_file_path):
        
        gene_set_x_sample = ccal.single_sample_gseas(
            gene_x_sample,
            gene_sets,
            file_path=gene_set_x_sample_file_path,
        )
    
    else:

        gene_set_x_sample = pd.read_table(
            gene_set_x_sample_file_path,
            index_col=0,
        )

    feature_dicts[name] = {
        'df': gene_set_x_sample,
        'data_type': 'continuous',
        'emphasis': 'high',
    }

In [ ]:
for gene_set_file_path in gene_set_file_paths:
    
    gene_sets = ccal.read_gmt(gene_set_file_path)
    
    make_gene_set_x_sample_and_add_to_feature_dicts(
        gene_set_file_path.split('/')[-1],
        gene_sets,
        OVERWRITE,
    )

In [ ]:
all_gene_sets = ccal.drop_df_slice(
    ccal.read_gmts(gene_set_file_paths),
    1,
    min_n_not_na_unique_value=3,
)

is_gene_set_to_peek = np.full(
    all_gene_sets.shape[0],
    False,
)

gene_set_names_lower = all_gene_sets.index.str.lower()

for gene_set in GENE_SETS_TO_PEEK:

    is_gene_set_to_peek |= gene_set_names_lower == gene_set.lower()

for gene_set_keyword in GENE_SET_KEYWORDS_TO_PEEK:

    is_gene_set_to_peek |= gene_set_names_lower.str.contains(gene_set_keyword)

gene_sets_to_peek = all_gene_sets.loc[is_gene_set_to_peek]

gene_sets_to_peek = gene_sets_to_peek.append(pd.Series(
    GENES_TO_PEEK,
    name='Genes to Peek',
    index=('Gene {}'.format(i) for i in range(len(GENES_TO_PEEK))),
))

make_gene_set_x_sample_and_add_to_feature_dicts(
    'Gene Sets to Peek',
    gene_sets_to_peek, 
    OVERWRITE,
)

In [ ]:
ccal.make_match_panels(
    target_x_sample,
    feature_dicts,
    drop_negative_target=True,
    target_ascending=True,
    n_job=N_JOB,
    n_required_for_match_function=0.5,
    extreme_feature_threshold=EXTREME_FEATURE_THRESHOLD,
    n_sampling=N_SAMPLING,
    n_permutation=N_PERMUTATION,
    target_type='binary',
    plot_features_std_max=PLOT_FEATURES_STD_MAX,
    directory_path=output_directory_path,
    overwrite=OVERWRITE,
)

In [ ]:
gmt_name = 'stem_cell.gmt'

gene_set_name = 'Cancer Stem Cell'

gene_set_genes = all_gene_sets.loc[gene_set_name].dropna()

df = pd.concat(
    (
        feature_dicts[gmt_name]['df'].loc[[gene_set_name]],
        gene_x_sample.loc[gene_set_genes],
    ),
)

n_target = target_x_sample.shape[0]

ccal.make_match_panels(
    (target_x_sample.iloc[i] for i in range(n_target)),
    (
        True,
    ) * n_target,
    (
        True,
    ) * n_target,
    (
        'binary',
    ) * n_target,
    {
        gene_set_name: {
            'df': df,
            'data_type': 'continuous',
            'emphasis': 'low',
        },
    },
    extreme_feature_threshold=None,
    n_sampling=N_SAMPLING,
    n_permutation=N_PERMUTATION,
    row_height=160,
    plot_features_std_max=PLOT_FEATURES_STD_MAX,
    directory_path=output_directory_path,
)

In [ ]:
title_feature_dicts = {
    'Hallmark Gene Sets': (
        {
            'feature_group': 'h.all.v6.1.symbols.gmt',
            'indices': (
                'HALLMARK_UV_RESPONSE_UP',
            ),
        },
        {
            'feature_group': 'h.all.v6.1.symbols.gmt',
            'indices': (
                'HALLMARK_OXIDATIVE_PHOSPHORYLATION',
                'HALLMARK_FATTY_ACID_METABOLISM',
            ),
        },
        {
            'feature_group': 'h.all.v6.1.symbols.gmt',
            'indices': (
                'HALLMARK_E2F_TARGETS',
            ),
        },
        {
            'feature_group': 'h.all.v6.1.symbols.gmt',
            'indices': (
                'HALLMARK_TGF_BETA_SIGNALING',
                'HALLMARK_NOTCH_SIGNALING',
                'HALLMARK_WNT_BETA_CATENIN_SIGNALING',
            ),
        },
    ),
    'Stemness Gene Sets (Main Figure)': (
        {
            'feature_group': 'stem_cell.gmt',
            'indices': (
                'Cancer Stem Cell',
            ),
            'emphasis': 'low',
        },
        {
            'feature_group': 'h.all.v6.1.symbols.gmt',
            'indices': (
                'HALLMARK_NOTCH_SIGNALING',
                'HALLMARK_TGF_BETA_SIGNALING',
                'HALLMARK_E2F_TARGETS',
                'HALLMARK_WNT_BETA_CATENIN_SIGNALING',
            ),
            'emphasis': 'low',
        },
    ),
    'Stemness Gene Sets (Supplementary Figure)': (
        {
            'feature_group': 'c2.all.v6.1.symbols.gmt',
            'indices': (
                'BENPORATH_ES_CORE_NINE',
                'BENPORATH_ES_WITH_H3K27ME3',
                'HOEBEKE_LYMPHOID_STEM_CELL_UP',
                'IVANOVA_HEMATOPOIESIS_STEM_CELL_AND_PROGENITOR',
                'MIKKELSEN_IPS_LCP_WITH_H3K4ME3',
                'MIKKELSEN_IPS_WITH_HCP_H3K27ME3',
                'GOTZMANN_EPITHELIAL_TO_MESENCHYMAL_TRANSITION_UP',
            ),
            'emphasis': 'low',
        },
        {
            'feature_group': 'c2.all.v6.1.symbols.gmt',
            'indices': (
                'BIOCARTA_WNT_PATHWAY',
            ),
            'emphasis': 'low',
        },
        {
            'feature_group': 'c6.all.v6.1.symbols.gmt',
            'indices': (
                'PRC2_EZH2_UP.V1_UP',
                'E2F3_UP.V1_UP',
            ),
            'emphasis': 'low',
        },
    ),
    'NFkB Gene Sets': (
        {
            'feature_group': 'c2.all.v6.1.symbols.gmt',
            'indices': (
                'HINATA_NFKB_TARGETS_KERATINOCYTE_UP',
                'TIAN_TNF_SIGNALING_VIA_NFKB',
            ),
        },
    ),
    'Differentiation Gene Sets': (
        {
            'feature_group': 'c2.all.v6.1.symbols.gmt',
            'indices': (
                'RODRIGUES_THYROID_CARCINOMA_POORLY_DIFFERENTIATED_UP',
                'RODRIGUES_THYROID_CARCINOMA_POORLY_DIFFERENTIATED_DN',
                'MA_MYELOID_DIFFERENTIATION_UP',
                'ADDYA_ERYTHROID_DIFFERENTIATION_BY_HEMIN',
            ),
        },
    ),
}

In [ ]:
for target_name, target in target_x_sample.iterrows():

    target = target[target != -1]
    
    highlight_directory_path = '{}/{}/highlight'.format(
        output_directory_path,
        target.name,
    )

    ccal.establish_path(
        highlight_directory_path,
        'directory',
    )
    
    for title, feature_dicts_ in title_feature_dicts.items():
        
        ccal.make_summary_match_panel(
            target,
            {i: {
                **feature_dicts[feature_dict['feature_group']],
                **feature_dict,
                'score': pd.read_table(
                    '{}/{}/{}.tsv'.format(
                        output_directory_path,
                        target.name,
                        feature_dict['feature_group'],
                    ),
                    index_col=0,
                ),
            } for i, feature_dict in enumerate(feature_dicts_)},
            target_ascending=None,
            target_type='binary',
            title='{}<br>{}'.format(
                target.name,
                title,
            ),
            row_height=160,
            plot_features_std_max=PLOT_FEATURES_STD_MAX,
            html_file_path='{}/{}.html'.format(
                highlight_directory_path,
                title,
            ),
        )

In [ ]:
from scipy.stats import pearsonr


def function(
    gene_x_sample_row,
    target,
):
   
    return pearsonr(
        gene_x_sample_row,
        target,
    )[0]

In [ ]:
for target_name, target in target_x_sample.iterrows():
    
    target = target[target != -1]
    
    mountain_plot_directory_path = '{}/{}/mountain_plot'.format(
        output_directory_path,
        target.name,
    )

    ccal.establish_path(
        mountain_plot_directory_path,
        'directory',
    )
    
    gene_x_target_sample = gene_x_sample[target.index]
    
    gene_x_target_sample = pd.DataFrame(
        ccal.normalize_nd_array(
            gene_x_target_sample[target.index].values,
            0,
            '-0-',
            raise_for_bad_value=False,
        ),
        index=gene_x_target_sample.index,
        columns=gene_x_target_sample.columns,
    )

    for gene_set_name, genes in gene_sets_to_peek.loc[[
        'Cancer Stem Cell',
        'Genes to Peek',
    ]].iterrows():

        score, p_value = ccal.gsea(
            gene_x_target_sample,
            target,
            genes,
            function,
            title='{} & {}'.format(
                target.name,
                genes.name,
            ),
            gene_score_name='Correlation',
            html_file_path='{}/{}.html'.format(
                mountain_plot_directory_path,
                genes.name,
            ),
        )